## Problem Statement
Given your experience working with the Aave V2 protocol, it's assumed that you have a clear understanding of lending protocols and on-chain transaction analysis.

#### NOTE: 
Using Covalent API to retrieve general Ethereum transactions for each wallet.

### Importing required libraries

In [1]:
import pandas as pd
import requests
from sklearn.preprocessing import MinMaxScaler

### Data Exploration
- Wallet transaction data was retrieved using the **Covalent API**, which provides historical Ethereum transactions per wallet.
- Only the wallet addresses given in the CSV were queried.
- Each transaction is tagged with the wallet address for reference after flattening the API response.


In [2]:
# Loading Wallets
wallets_id = pd.read_csv('Wallet id - Sheet1.csv')
wallets_id

,wallet_id
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,0x06b51c6882b27cb05e712185531c1f74996dd988
2,0x0795732aacc448030ef374374eaae57d2965c16c
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
...,...
98,0xf60304b534f74977e159b2e159e135475c245526
99,0xf67e8e5805835465f7eba988259db882ab726800
100,0xf7aa5d0752cfcd41b0a5945867d619a80c405e52
101,0xf80a8b9cfff0febf49914c269fb8aead4a22f847


In [3]:
# Converting wallets to list
wallets_list = wallets_id['wallet_id'].tolist()
print(wallets_list[:5])

['0x0039f22efb07a647557c7c5d17854cfd6d489ef3', '0x06b51c6882b27cb05e712185531c1f74996dd988', '0x0795732aacc448030ef374374eaae57d2965c16c', '0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9', '0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae']


#### Fetching the Transactions History from API

In [4]:
API_KEY = 'cqt_rQ7mB4WDDqgyTfhT6QxptKRyx8FF'

In [5]:
all_txns = []
for wallet in wallets_list:
    url = f"https://api.covalenthq.com/v1/1/address/{wallet}/transactions_v2/?key={API_KEY}"
    try:
        response = requests.get(url)
        data = response.json()
        txns = data["data"]["items"]
        for txn in txns:
            txn["wallet"] = wallet
            all_txns.append(txn)
    except Exception as e:
        print(f"Fetching Failed: {e}")

In [27]:
# Displaying raw transaction
all_txns[:1]

[{'block_signed_at': '2025-06-16T21:15:11Z',
  'block_height': 22719696,
  'block_hash': '0xff57a0c234e73c4897632e198caa0074e84d16080e0f99731a3da891faebb42b',
  'tx_hash': '0x98703fb4a7c6804d82e98f009ecc0e089abd53de94696088fb9675dde740c570',
  'tx_offset': 108,
  'successful': True,
  'miner_address': '0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5',
  'from_address': '0xc6b602de080fc9ac9d96a431b2d749d38e77cbbc',
  'from_address_label': None,
  'to_address': '0x13173761e24c3708495b1dd314920f67f97011d0',
  'to_address_label': None,
  'value': '0',
  'value_quote': 0.0,
  'pretty_value_quote': '$0.00',
  'gas_metadata': {'contract_decimals': 18,
   'contract_name': 'Ether',
   'contract_ticker_symbol': 'ETH',
   'contract_address': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee',
   'supports_erc': None,
   'logo_url': 'https://www.datocms-assets.com/86369/1669619533-ethereum.png'},
  'gas_offered': 84929,
  'gas_spent': 55815,
  'gas_price': 4200000000,
  'fees_paid': '234423000000000',
  'gas

In [9]:
# Converting 'all_txns' list to Dataframe
txns = pd.DataFrame(all_txns)
txns.head()

,block_signed_at,block_height,block_hash,tx_hash,tx_offset,successful,miner_address,from_address,from_address_label,to_address,...,gas_metadata,gas_offered,gas_spent,gas_price,fees_paid,gas_quote,pretty_gas_quote,gas_quote_rate,log_events,wallet
0,2025-06-16T21:15:11Z,22719696,0xff57a0c234e73c4897632e198caa0074e84d16080e0f...,0x98703fb4a7c6804d82e98f009ecc0e089abd53de9469...,108,True,0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5,0xc6b602de080fc9ac9d96a431b2d749d38e77cbbc,None,0x13173761e24c3708495b1dd314920f67f97011d0,...,"{'contract_decimals': 18, 'contract_name': 'Et...",84929,55815,4200000000,234423000000000,0.599307,$0.60,2556.520508,"[{'block_signed_at': '2025-06-16T21:15:11Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,2025-04-27T21:09:35Z,22362988,0xe8a130e1e7110fbdde7dd51e74d940f538057be01240...,0x0b8e89905717b9babd69aa7db6af0afb83331e9b8041...,131,True,0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5,0xfd0ce77ca6521e3cbbcfb07398cca8ec2648115d,None,None,...,"{'contract_decimals': 18, 'contract_name': 'Et...",5500000,5264347,500000000,2632173500000000,4.722150,$4.72,1794.011719,"[{'block_signed_at': '2025-04-27T21:09:35Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3
2,2025-03-24T23:27:47Z,22120041,0x893f17102836ef520b5239cf68595b7c02c78e206582...,0x5745f6d2e9397ed8ceb7120f42761e1f04643e132e94...,150,True,0x4838b106fce9647bdf1e7877bf73ce8b0bad5f97,0xc07aff7c831105bacaf2fb4d9506332112eee2a1,None,None,...,"{'contract_decimals': 18, 'contract_name': 'Et...",5500000,5226097,700000000,3658267900000000,7.601623,$7.60,2077.929443,"[{'block_signed_at': '2025-03-24T23:27:47Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3
3,2025-02-23T19:08:59Z,21911013,0x016232bca14eaedba1297086544e1a24a3868ce8d40b...,0x35cd404392ac9b7786bee78e4bfb2773cbb576b9c3dc...,107,True,0x4838b106fce9647bdf1e7877bf73ce8b0bad5f97,0xad38352c725c6768c11a9a71bc7d921a9355d56c,None,None,...,"{'contract_decimals': 18, 'contract_name': 'Et...",5500000,5213387,950000000,4952717650000000,14.001496,$14.00,2827.032959,"[{'block_signed_at': '2025-02-23T19:08:59Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3
4,2025-02-11T03:05:11Z,21820480,0x385f706cfcfdf648c9196d4120011c354371bd1977db...,0x51a4f04abaa498409b72d83b4365440f0738ac5bcca1...,196,True,0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5,0x1e6971036be043c0e2457fe3f6dbed9f73354c6b,None,None,...,"{'contract_decimals': 18, 'contract_name': 'Et...",5500000,5244910,1200000000,6293892000000000,16.379029,$16.38,2602.368896,"[{'block_signed_at': '2025-02-11T03:05:11Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3


In [10]:
# Filtering transactions to include only those related to the wallet addresses we have
wallet_list = wallets_id.iloc[:, 0].tolist()
df = txns[txns['wallet'].isin(wallet_list)]

In [11]:
df.head()

,block_signed_at,block_height,block_hash,tx_hash,tx_offset,successful,miner_address,from_address,from_address_label,to_address,...,gas_metadata,gas_offered,gas_spent,gas_price,fees_paid,gas_quote,pretty_gas_quote,gas_quote_rate,log_events,wallet
0,2025-06-16T21:15:11Z,22719696,0xff57a0c234e73c4897632e198caa0074e84d16080e0f...,0x98703fb4a7c6804d82e98f009ecc0e089abd53de9469...,108,True,0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5,0xc6b602de080fc9ac9d96a431b2d749d38e77cbbc,None,0x13173761e24c3708495b1dd314920f67f97011d0,...,"{'contract_decimals': 18, 'contract_name': 'Et...",84929,55815,4200000000,234423000000000,0.599307,$0.60,2556.520508,"[{'block_signed_at': '2025-06-16T21:15:11Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,2025-04-27T21:09:35Z,22362988,0xe8a130e1e7110fbdde7dd51e74d940f538057be01240...,0x0b8e89905717b9babd69aa7db6af0afb83331e9b8041...,131,True,0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5,0xfd0ce77ca6521e3cbbcfb07398cca8ec2648115d,None,None,...,"{'contract_decimals': 18, 'contract_name': 'Et...",5500000,5264347,500000000,2632173500000000,4.722150,$4.72,1794.011719,"[{'block_signed_at': '2025-04-27T21:09:35Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3
2,2025-03-24T23:27:47Z,22120041,0x893f17102836ef520b5239cf68595b7c02c78e206582...,0x5745f6d2e9397ed8ceb7120f42761e1f04643e132e94...,150,True,0x4838b106fce9647bdf1e7877bf73ce8b0bad5f97,0xc07aff7c831105bacaf2fb4d9506332112eee2a1,None,None,...,"{'contract_decimals': 18, 'contract_name': 'Et...",5500000,5226097,700000000,3658267900000000,7.601623,$7.60,2077.929443,"[{'block_signed_at': '2025-03-24T23:27:47Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3
3,2025-02-23T19:08:59Z,21911013,0x016232bca14eaedba1297086544e1a24a3868ce8d40b...,0x35cd404392ac9b7786bee78e4bfb2773cbb576b9c3dc...,107,True,0x4838b106fce9647bdf1e7877bf73ce8b0bad5f97,0xad38352c725c6768c11a9a71bc7d921a9355d56c,None,None,...,"{'contract_decimals': 18, 'contract_name': 'Et...",5500000,5213387,950000000,4952717650000000,14.001496,$14.00,2827.032959,"[{'block_signed_at': '2025-02-23T19:08:59Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3
4,2025-02-11T03:05:11Z,21820480,0x385f706cfcfdf648c9196d4120011c354371bd1977db...,0x51a4f04abaa498409b72d83b4365440f0738ac5bcca1...,196,True,0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5,0x1e6971036be043c0e2457fe3f6dbed9f73354c6b,None,None,...,"{'contract_decimals': 18, 'contract_name': 'Et...",5500000,5244910,1200000000,6293892000000000,16.379029,$16.38,2602.368896,"[{'block_signed_at': '2025-02-11T03:05:11Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3


In [12]:
df.shape

(2823, 24)

In [13]:
df.info

<bound method DataFrame.info of            block_signed_at  block_height  \
0     2025-06-16T21:15:11Z      22719696   
1     2025-04-27T21:09:35Z      22362988   
2     2025-03-24T23:27:47Z      22120041   
3     2025-02-23T19:08:59Z      21911013   
4     2025-02-11T03:05:11Z      21820480   
...                    ...           ...   
2818  2020-07-29T06:16:50Z      10552810   
2819  2020-10-21T18:51:46Z      11101237   
2820  2020-10-21T18:51:31Z      11101235   
2821  2020-10-21T11:57:13Z      11099370   
2822  2020-10-21T11:56:05Z      11099365   

                                             block_hash  \
0     0xff57a0c234e73c4897632e198caa0074e84d16080e0f...   
1     0xe8a130e1e7110fbdde7dd51e74d940f538057be01240...   
2     0x893f17102836ef520b5239cf68595b7c02c78e206582...   
3     0x016232bca14eaedba1297086544e1a24a3868ce8d40b...   
4     0x385f706cfcfdf648c9196d4120011c354371bd1977db...   
...                                                 ...   
2818  0xaddfaf8d626c475d73f6c7f1e1cb248634595c089356...   
2819  0xff19e5498ca75ca9170f6ef489d8b8ca6716a6a60b70...   
2820  0x14382455c69542f79d9541f6528d661efb67fbe7a78c...   
2821  0x057fa26377a5d0bb608d04789d6728a038c9d186477b...   
2822  0x08fa7d902261df4a83ace02adbc7f4c9b8aa667c0303...   

                                                tx_hash  tx_offset  \
0     0x98703fb4a7c6804d82e98f009ecc0e089abd53de9469...        108   
1     0x0b8e89905717b9babd69aa7db6af0afb83331e9b8041...        131   
2     0x5745f6d2e9397ed8ceb7120f42761e1f04643e132e94...        150   
3     0x35cd404392ac9b7786bee78e4bfb2773cbb576b9c3dc...        107   
4     0x51a4f04abaa498409b72d83b4365440f0738ac5bcca1...        196   
...                                                 ...        ...   
2818  0x55030001bba1945f08497ff31ea3e836ba48b68b4726...        198   
2819  0xe76b49f4a25d49d8e5279e905ca683488f1a21313e57...         50   
2820  0x5a10a092184342b7e2132f7a60868a735c78b85445ed...        142   
2821  0xf2135f7e2a3490c5706f36def69926a5b3ae71140bab...        159   
2822  0x9c2708de3fa45580cb33d08d144b3888b4f360f416f0...         54   

      successful                               miner_address  \
0           True  0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5   
1           True  0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5   
2           True  0x4838b106fce9647bdf1e7877bf73ce8b0bad5f97   
3           True  0x4838b106fce9647bdf1e7877bf73ce8b0bad5f97   
4           True  0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5   
...          ...                                         ...   
2818        True  0x5a0b54d5dc17e0aadc383d2db43b0a0d3e029c4c   
2819        True  0xea674fdde714fd979de3edf0f56aa9716b898ec8   
2820        True  0x5a0b54d5dc17e0aadc383d2db43b0a0d3e029c4c   
2821        True  0x45ddbabcf2cce1aadbd04f5b8745579f3aa2949c   
2822        True  0x5a0b54d5dc17e0aadc383d2db43b0a0d3e029c4c   

                                    from_address from_address_label  \
0     0xc6b602de080fc9ac9d96a431b2d749d38e77cbbc               None   
1     0xfd0ce77ca6521e3cbbcfb07398cca8ec2648115d               None   
2     0xc07aff7c831105bacaf2fb4d9506332112eee2a1               None   
3     0xad38352c725c6768c11a9a71bc7d921a9355d56c               None   
4     0x1e6971036be043c0e2457fe3f6dbed9f73354c6b               None   
...                                          ...                ...   
2818  0xfdcab1b4e6292ddd540a3c535f36d32d879aa78a               None   
2819  0xfe5a05c0f8b24fca15a7306f6a4ebb7dcf2186ac               None   
2820  0xfe5a05c0f8b24fca15a7306f6a4ebb7dcf2186ac               None   
2821  0x73008d40648d3ccd191d9af6889159c15089095d               None   
2822  0x9c88429c3f1e701cfe5fe4c8536f90440fac6c47               None   

                                      to_address  ...  \
0     0x13173761e24c3708495b1dd314920f67f97011d0  ...   
1                                           None  ...   
2                                           None  ...   
3                                      

From the above data info:
- The 'block_signed_at column' is in text format, but it contains dates. We need to convert it to datetime to work with date-related features.
- The value column is also in text format. Since it shows the transaction amount, we convert it to float to do calculations.
- The gas_spent column is in integer format. We convert it to float to keep all numeric columns consistent.

In [14]:
# Convert to datetime
df['block_signed_at'] = pd.to_datetime(df['block_signed_at'])

# Convert to float
df['value'] = df['value'].astype(float)
df['gas_spent'] = df['gas_spent'].astype(float)

In [15]:
# Display all columns
pd.set_option('display.max_columns', None)

In [16]:
df.head()

,block_signed_at,block_height,block_hash,tx_hash,tx_offset,successful,miner_address,from_address,from_address_label,to_address,to_address_label,value,value_quote,pretty_value_quote,gas_metadata,gas_offered,gas_spent,gas_price,fees_paid,gas_quote,pretty_gas_quote,gas_quote_rate,log_events,wallet
0,2025-06-16 21:15:11+00:00,22719696,0xff57a0c234e73c4897632e198caa0074e84d16080e0f...,0x98703fb4a7c6804d82e98f009ecc0e089abd53de9469...,108,True,0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5,0xc6b602de080fc9ac9d96a431b2d749d38e77cbbc,None,0x13173761e24c3708495b1dd314920f67f97011d0,None,0.0,0.0,$0.00,"{'contract_decimals': 18, 'contract_name': 'Et...",84929,55815.0,4200000000,234423000000000,0.599307,$0.60,2556.520508,"[{'block_signed_at': '2025-06-16T21:15:11Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,2025-04-27 21:09:35+00:00,22362988,0xe8a130e1e7110fbdde7dd51e74d940f538057be01240...,0x0b8e89905717b9babd69aa7db6af0afb83331e9b8041...,131,True,0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5,0xfd0ce77ca6521e3cbbcfb07398cca8ec2648115d,None,None,None,0.0,0.0,$0.00,"{'contract_decimals': 18, 'contract_name': 'Et...",5500000,5264347.0,500000000,2632173500000000,4.722150,$4.72,1794.011719,"[{'block_signed_at': '2025-04-27T21:09:35Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3
2,2025-03-24 23:27:47+00:00,22120041,0x893f17102836ef520b5239cf68595b7c02c78e206582...,0x5745f6d2e9397ed8ceb7120f42761e1f04643e132e94...,150,True,0x4838b106fce9647bdf1e7877bf73ce8b0bad5f97,0xc07aff7c831105bacaf2fb4d9506332112eee2a1,None,None,None,0.0,0.0,$0.00,"{'contract_decimals': 18, 'contract_name': 'Et...",5500000,5226097.0,700000000,3658267900000000,7.601623,$7.60,2077.929443,"[{'block_signed_at': '2025-03-24T23:27:47Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3
3,2025-02-23 19:08:59+00:00,21911013,0x016232bca14eaedba1297086544e1a24a3868ce8d40b...,0x35cd404392ac9b7786bee78e4bfb2773cbb576b9c3dc...,107,True,0x4838b106fce9647bdf1e7877bf73ce8b0bad5f97,0xad38352c725c6768c11a9a71bc7d921a9355d56c,None,None,None,0.0,0.0,$0.00,"{'contract_decimals': 18, 'contract_name': 'Et...",5500000,5213387.0,950000000,4952717650000000,14.001496,$14.00,2827.032959,"[{'block_signed_at': '2025-02-23T19:08:59Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3
4,2025-02-11 03:05:11+00:00,21820480,0x385f706cfcfdf648c9196d4120011c354371bd1977db...,0x51a4f04abaa498409b72d83b4365440f0738ac5bcca1...,196,True,0x95222290dd7278aa3ddd389cc1e1d165cc4bafe5,0x1e6971036be043c0e2457fe3f6dbed9f73354c6b,None,None,None,0.0,0.0,$0.00,"{'contract_decimals': 18, 'contract_name': 'Et...",5500000,5244910.0,1200000000,6293892000000000,16.379029,$16.38,2602.368896,"[{'block_signed_at': '2025-02-11T03:05:11Z', '...",0x0039f22efb07a647557c7c5d17854cfd6d489ef3


### Feature Engineering

The following features were selected as risk indicators:
- **txn_count**: Higher transaction count may indicate more activity or less dormant wallets.
- **total_value_sent / received**: Helps understand the volume of funds handled.
- **active_days**: Measures consistent engagement over time.
- **duration_days**: Long activity duration may correlate with more reliable wallet behavior.
- **avg_txn_value**: Captures transaction size patterns, useful for outlier detection.

In [17]:
# Grouping by wallet and computing features
features = df.groupby('wallet').agg(
    txn_count=('tx_hash', 'count'),
    total_value_sent=('value', lambda x: x[df['from_address'] == df['wallet']].sum()),
    total_value_received=('value', lambda x: x[df['to_address'] == df['wallet']].sum()),
    total_gas_spent=('gas_spent', 'sum'),
    avg_txn_value=('value', 'mean'),
    active_days=('block_signed_at', lambda x: x.dt.date.nunique()),
    first_txn_date=('block_signed_at', 'min'),
    last_txn_date=('block_signed_at', 'max'),
).reset_index()

# Finding Duration
features['duration_days'] = (features['last_txn_date'] - features['first_txn_date']).dt.days

In [18]:
features.head()

,wallet,txn_count,total_value_sent,total_value_received,total_gas_spent,avg_txn_value,active_days,first_txn_date,last_txn_date,duration_days
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,100,5.225464e+19,0.000000e+00,235826553.0,5.225464e+17,77,2023-04-14 19:48:35+00:00,2025-06-16 21:15:11+00:00,794
1,0x06b51c6882b27cb05e712185531c1f74996dd988,5,5.701850e+15,1.860000e+16,308732.0,4.860370e+15,1,2020-10-16 00:12:29+00:00,2020-10-16 00:19:32+00:00,0
2,0x0795732aacc448030ef374374eaae57d2965c16c,4,0.000000e+00,1.860000e+16,290793.0,4.650000e+15,1,2020-11-25 12:40:49+00:00,2020-11-25 13:13:37+00:00,0
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,80,1.351075e+20,6.667600e+19,132280277.0,2.523044e+18,57,2019-07-28 05:10:56+00:00,2025-03-25 19:18:59+00:00,2067
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,5,7.404691e+15,1.860000e+16,308720.0,5.200938e+15,2,2020-10-21 12:37:23+00:00,2020-12-24 09:47:49+00:00,63


### Data Preprocessing

In [19]:
# Feature Scaling(MinMax Scaling)
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features[['txn_count', 'total_value_sent', 'total_value_received',
                                                 'total_gas_spent', 'avg_txn_value', 'active_days', 'duration_days']])

In [20]:
scaled_features

array([[1.00000000e+00, 7.60203724e-02, 0.00000000e+00, 7.02143178e-01,
        3.33275660e-02, 1.00000000e+00, 2.77428372e-01],
       [2.06185567e-02, 8.29508632e-06, 4.53503072e-05, 3.26095268e-04,
        3.09990266e-04, 0.00000000e+00, 0.00000000e+00],
       [1.03092784e-02, 0.00000000e+00, 4.53503072e-05, 2.72639030e-04,
        2.96573047e-04, 0.00000000e+00, 0.00000000e+00],
       [7.93814433e-01, 1.96555212e-01, 1.62568660e-01, 3.93586717e-01,
        1.60917581e-01, 7.36842105e-01, 7.22222222e-01],
       [2.06185567e-02, 1.07723898e-05, 4.53503072e-05, 3.26059509e-04,
        3.31711417e-04, 1.31578947e-02, 2.20125786e-02],
       [3.09278351e-02, 6.90825612e-04, 1.18502830e-03, 4.72684972e-04,
        1.02140785e-02, 2.63157895e-02, 2.43885395e-01],
       [3.09278351e-02, 1.51491741e-05, 4.53503072e-05, 8.08375485e-04,
        3.08406109e-04, 0.00000000e+00, 0.00000000e+00],
       [4.12371134e-02, 2.28004112e-05, 4.53503072e-05, 1.17731560e-03,
        3.12267036e-04, 0

In [21]:
# Converting to DataFrame
features_scaled = pd.DataFrame(scaled_features,columns=[
    'txn_count', 'total_value_sent', 'total_value_received',
    'total_gas_spent', 'avg_txn_value', 'active_days', 'duration_days'
])

### Risk Scoring Method

- All numeric features were normalized using **MinMaxScaler** to bring them into a comparable range (0–1).
- Each normalized feature was assigned **equal weight (0.2)**.
- Final score = Weighted sum × 1000 to scale into 0–1000 range.

In [22]:
# Assign weights based on feature importance.
features['credit_score'] = (
    features_scaled['txn_count'] * 0.2 +
    features_scaled['total_value_sent'] * 0.2 +
    features_scaled['total_value_received'] * 0.2 +
    features_scaled['active_days'] * 0.2 +
    features_scaled['duration_days'] * 0.2
) * 1000

In [23]:
# Saving final scores
features[['wallet', 'credit_score']].to_csv('wallet_credit_scores.csv', index=False)

In [24]:
# Loading wallet scores
wallet_scores = pd.read_csv('wallet_credit_scores.csv')
wallet_scores.head()

,wallet,credit_score
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,470.689749
1,0x06b51c6882b27cb05e712185531c1f74996dd988,4.134440
2,0x0795732aacc448030ef374374eaae57d2965c16c,2.070926
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,522.400526
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,11.169031


In [25]:
# Displaying top 10 wallets
top_wallets = features.sort_values(by='credit_score', ascending=False).head(10)
print(top_wallets[['wallet', 'credit_score']])

                                        wallet  credit_score
43  0x623af911f493747c216ad389c7805a37019c662d    862.801360
3   0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9    522.400526
0   0x0039f22efb07a647557c7c5d17854cfd6d489ef3    470.689749
80  0xbd4a00764217c13a246f86db58d74541a0c3972a    465.394916
21  0x330513970efd9e8dd606275fb4c50378989b3204    455.685822
26  0x427f2ac5fdf4245e027d767e7c3ac272a1f40a65    435.718544
90  0xdde73df7bd4d704a89ad8421402701b3a460c6e9    430.535269
30  0x4d997c89bc659a3e8452038a8101161e7e7e53a7    420.229149
48  0x70d8e4ab175dfe0eab4e9a7f33e0a2d19f44001e    416.381214
22  0x3361bea43c2f5f963f81ac70f64e6fba1f1d2a97    415.150091


### Final Deliverables

- `wallet_credit_scores.csv`: A file containing wallet IDs and their corresponding risk scores between 0–1000.
- Risk scoring is based on wallet-level transaction behavior including transaction count, gas used, value transferred, and activity duration.
